Load the necessary packages.  Uncomment the first lines if it is necessary to add or update the packages.

In [ ]:
#using Pkg
#Pkg.add("https://github.com/DanBerge/RunOrderOptim")
#Pkg.add(["Convex","GLPKMathProgInterface", "DataFrames"])

using RunOrderOptim, Convex, GLPKMathProgInterface

Load the class information for the event.

In [9]:
df_classes=RunOrderOptim.read_classes()
first(df_classes,12) # Truncate the view for display purposes

#show(df_classes,allrows=true) # Uncomment to show all the classes

,Class,ClassGroup,BumpClass
,String,String,String⍰
1,SS,Street,C1
2,AS,Street,C1
3,BS,Street,C1
4,CS,Street,C1
5,DS,Street,C1
6,ES,Street,C1
7,FS,Street,C1
8,GS,Street,C1
9,HS,Street,C1


Load the entries for event from MotorSportsReg.com export

In [10]:
df_entries=RunOrderOptim.read_entries() #Default to example event
first(df_entries,12) # Truncate the view for display purposes
#show(df_entries,allrows=true) # Uncomment to show all the classes

,LastName,FirstName,Class,Index,Exempt,Novice,IndexedClass
,String,String,String,String⍰,Bool,Bool,String
1,A,David,NA,missing,false,true,NA
2,A,Dwight,HS,Z,false,false,Z
3,A,Kerry,STS,P,true,false,P
4,A,Erik,GS,missing,false,false,GS
5,B,Dave,FS,Z,false,false,Z
6,B,Adam,STS,P,false,false,P
7,B,Drew,ASP,Z,false,false,Z
8,B,Robert,NB,missing,false,true,NB
9,B,Jody,SSC,missing,false,false,SSC


Set up the Convex optimization problem using the event classes and entries.
All of the available options and 

In [29]:
prob=RunOrderOptim.setup_problem(df_classes,df_entries;
    separate_classes=["Z"=>"P"], #Keep pairs of classes classes P & Z separate.  This may make the problem infesible
    
#### Additional Options with default values    
    run_groups=2, #Numer of run groups
    min_class_size=4, #Minimum size for a class to trophy, otherwise place in bump class
    keep_empty=false, #Preserve empty classes
    workerweight=10, #Weight of workers used in optimization
    driverweight=2,  #Weight of drivers used in optimization
    noviceweight=1)  #Weight of novice drivers used in optimization
    
prob.x #Show the binary selection variable

┌ Info: Combining CAMC, CAMS, CAMT, EP, XP, SSC, DSP, STS, STU
└ @ RunOrderOptim H:\Projects\Julia\RunOrderOptim\src\RunOrderOptim.jl:53
┌ Info: Combining CS, FS, GS, HS, SS
└ @ RunOrderOptim H:\Projects\Julia\RunOrderOptim\src\RunOrderOptim.jl:53


Variable of
size: (51, 2)
sign: NoSign()
vexity: AffineVexity()

Solve the problem with with the selected solver.

In [20]:
solve!(prob.problem,GLPKSolverMIP()) #Solve the problem using the GLPK Mixed Interger solver

In [21]:
prob.problem.status #Check the status of the solution

:Optimal

The solver finds the minimum of the following objective function.  Check the result of the optimization.

optval = 10*(maximum(Workers)-minimum(Workers)) + 2*(maximum(Drivers)-minimum(Drivers)) + 1*(maximum(Novice)- minimum(Novice))

In [22]:
prob.problem.optval #Solution score.  Lower is better

13.0

In [23]:
df=RunOrderOptim.compact_solution(prob)

,RunGroup,Workers,Drivers,Novice
,Int64,Int64,Int64,Int64
1,1,48,51,11
2,2,47,51,10


In [24]:
RunOrderOptim.full_solution(prob)

,RunGroup,Class,ClassGroup,Drivers,Workers
,Int64,String,String,Int64,Int64
1,1,DM,Modified,4,4
2,1,P,Pro,23,10
3,1,DS,Street,10,9
4,1,SMF,Street Modified,5,5
5,1,SMF,Street Modified,5,5
6,1,SSM,Street Modified,4,4
7,1,Novice A-M,Novice,11,11
8,2,CAMC,Classic American,2,2
9,2,CAMS,Classic American,1,1


In [25]:
show(RunOrderOptim.full_solution(prob), allrows=true)

27×5 DataFrame
│ Row │ RunGroup │ Class      │ ClassGroup       │ Drivers │ Workers │
│     │ Int64    │ String     │ String           │ Int64   │ Int64   │
├─────┼──────────┼────────────┼──────────────────┼─────────┼─────────┤
│ 1   │ 1        │ DM         │ Modified         │ 4       │ 4       │
│ 2   │ 1        │ P          │ Pro              │ 23      │ 10      │
│ 3   │ 1        │ DS         │ Street           │ 10      │ 9       │
│ 4   │ 1        │ SMF        │ Street Modified  │ 5       │ 5       │
│ 5   │ 1        │ SMF        │ Street Modified  │ 5       │ 5       │
│ 6   │ 1        │ SSM        │ Street Modified  │ 4       │ 4       │
│ 7   │ 1        │ Novice A-M │ Novice           │ 11      │ 11      │
│ 8   │ 2        │ CAMC       │ Classic American │ 2       │ 2       │
│ 9   │ 2        │ CAMS       │ Classic American │ 1       │ 1       │
│ 10  │ 2        │ CAMT       │ Classic American │ 1       │ 0       │
│ 11  │ 2        │ X          │ Exhibition       │ 1       │ 1